# Elixhauser Comorbidity Score


This calculation of the Elixhauser Comorbidity Index has been designed to work on the OMOP CDM v5.3 and developed in the University of North Carolina at Chapel Hill de-identified OMOP Research Data Repository, ORDR(D). Please see the README in the repo with important notes, clarifications, and assumptions.

Author: Josh Fuchs

Copyright 2025, The University of North Carolina at Chapel Hill. Permission is granted to use in accordance with the MIT license. The code is licensed under the open-source MIT license.

# Load packages, connect to the database, define functions

In [22]:
import sqlalchemy
import psycopg
import pandas as pd
import numpy as np

from getpass import getpass
from itertools import groupby

In [23]:
# prompt for username and password
user_name = input('enter username:')
user_password = getpass('enter password:')


## configure connection
conn = psycopg.connect(
    host='od2-primary',
    dbname='ordrd',
    user=user_name,
    password=user_password)

enter username: jtfuchs
enter password: ········


In [24]:
cursor = conn.cursor()

In [25]:
def query_to_dataframe(query_string, column_names=None):
    '''
    Runs a SQL query on the database and returns the results
    as a pandas DataFrame.
    
    PARAMETERS:
    query_string : string
        string formatted as a SQL command, to execute on the database
        
    column_names : list, default=None
        list of strings for desired names of returned columns
        length must match the number of columns returned in the
        query        
        
        If you are selecting all (*) columns from a single table,
        this value can be None. In that case, this function will
        query the table to get the column names and use those to
        create the dataframe
    
    RETURNS:
        conditional. If error with query, returns
        
        query_result : None
        
        if query is successful, returns
        
        query_df : pandas dataframe
            DataFrame with column names set from column_names
    '''
    try:
        cursor.execute(query_string)
        query_result = cursor.fetchall()
    except:
        print("Error with SQL command")
        conn.rollback()
        query_result = None    

    if query_result is not None:
        query_df = pd.DataFrame(query_result,columns=column_names)
        return query_df
    
    return query_result

In [26]:
def split_list_to_dict(lst):
    """
    Splits a Python list into a dictionary where the keys are the length of each entry
    and the values are the corresponding entries.
    
    PARAMETERS:
    lst : list
        list of strings
        
    RETURNS
    result_dict : dictionary
        dictionary where keys are the length of entries in lst, and values are entries
        in lst that correspond to that length
    """
    result_dict = {}
    for item in lst:
        key = len(item)
        if key not in result_dict:
            result_dict[key] = []
        result_dict[key].append(item)
    return result_dict

In [42]:
def add_new_row(df,duplicate_dict):
    """
    There are some conditions that are listed with different levels of specificity 
    (i.e. I11 and I11.0) in Quan et al. (2005). This function identifies the long 
    version of the conditions in the dataframe (like I11.0), and adds in rows with the shorter
    condition (like I11). 
    
    For example, if both I11.0 and I11 are listed as conditions, I11.0 will be
    in the returned query from the database and this function will add in I11.  
        
    PARAMETERS
    df : pandas DataFrame
        DataFrame with condition_source_value a required column. This column should
        contain ICD-9 or -10 values. 
    
    duplicate_dict : dictionary
        dictionary containing mapping and strings to search for. Keys are 
        shorter versions of the condition, values are a list of strings
        that are the longer corresponding conditions. 
        
    RETURNS
    new_df : pandas DataFrame
        new rows are appended at the bottom of the input dataframe
    """

    for index, key in enumerate(duplicate_dict):
        # Identify rows where condition_source_value is in the list of values for that key
        mask = df['condition_source_value'].isin(duplicate_dict[key])
        # Create a new DataFrame with the same person_id and condition_source_value = key
        new_row = df[mask].copy()
        new_row['condition_source_value'] = key
        # Append the new row to the original DataFrame
        if index == 0:
            new_df = pd.concat([df,new_row],ignore_index=True)
        else:
            new_df = pd.concat([new_df,new_row],ignore_index=True)
    return new_df


In [28]:
def filter_related_conditions(df, serious_condition_list, less_serious_condition_list):
    """
    Filters the 'condition_source_value' column in a dataframe based on 'serious_condition_list' 
    and 'less_serious_condition_list' lists. If any 'serious_condition_list' values in row, then
    all 'less_serious_condition_list' values removed
    
    Args:
    df : DataFrame 
        DataFrame with a column 'condition_source_value' containing lists of ICD codes.
    
    serious_condition_list : list
        List of ICD codes of more serious related condition
    
    good_diabetes : list
        List of ICD codes of less serious related condtion. To be removed if needed.
    
    Returns:
        DataFrame: DataFrame with the filtered 'condition_source_value' column.
    """
    # Check if any elements in 'condition_source_value' are in 'serious_condition_list'
    mask_bad = df['condition_source_value'].apply(lambda x: any(code in serious_condition_list for code in x))
    
    # If any serious_condition_list codes are found, remove all 'less_serious_condition_list' codes
    df.loc[mask_bad, 'condition_source_value'] = df.loc[mask_bad, 'condition_source_value'].apply(
        lambda x: [code for code in x if code not in less_serious_condition_list]
    )
    
    return df

In [29]:
def calculate_score(condition_list,condition_df):
    """
    Calculates the Elixhauser Index Score given a list of conditions. If an ICD code appears 
    twice in the condition_df for different conditions, then it will be counted twice. 
    
    PARAMETERS
    condition_list : list
        list of conditions a patient has diagnoses for
        
    condition_df : DataFrame
        pandas dataframe of conditions, condition codes, and weights
        Must have a column named 'icd' with a list of ICD codes for each
        condition. 
        
    RETURNS
    total_score : int
        Elixhauser for these conditions
    """
    
    # Iterate through the condition_df to go through,
    # condition by condition, and see if any values in the
    # condition list match any of the values for that particular
    # condition. If so, sum the score for that condition to 
    # the overall score
    
    condition_set = set(condition_list)
    condition_df['icd_set'] = condition_df['icd'].apply(set)
    condition_df['intersection'] = condition_df['icd_set'].apply(lambda x: bool(condition_set.intersection(x)))
    total_score = condition_df.loc[condition_df['intersection'], 'weight'].sum()
    return total_score

# Build DataFrame for Conditions

We'll build a pandas DataFrame that will store ICD codes and weights for each Elixhauser condition. 

First, create a dictionary with conditions, ICD codes, and weights.

ICD codes: These are both ICD-9 and ICD-10 codes from Quan et al. (2005). These have been stripped of periods
to make matching easier in the database. 

Weights: These weights are from van Walraven et al. (2009)

In [30]:

columns = ['icd','weight']

condition_dictionary = {'congestive heart failure' : [['39891','40201','40211','40291','40401','40403','40411','40413',
                                                       '40491','40493','4254','4255','4256','4257','4258','4259','428',
                                                       'I099','I110','I130','I132','I255','I420','I425','I426','I427',
                                                       'I428','I429','P290','I43','I50'],
                                                      7],
                        'cardiac arrhythmias' : [['I441','I442','I443','I456','I459','I47','I48','I49','R000','R001','R008','T821','Z450','Z950',
                                                  '4260','42613','4267','4269','42610','42612','4270','4271','4272','4273','4274','4276','4277',
                                                  '4278','4279','7850','99601','99604','V450','V533'],5],
                        'valvular disease' : [['A520','I05','I06','I07','I08','I091','I098','I34','I35','I36','I37','I38','I39','Q230',
                                               'Q231','Q232','Q233','Z952','Z953','Z954','0932','394','395','396','397','424','7463',
                                               '7464','7465','7466','V422','V433'],
                                              -1],
                        'pulmonary circulation disorders' : [['I26','I27','I280','I288','I289','4150','4151','416','4170','4178','4179'],
                                                             4],
                        'peripheral vascular disorders' : [['I70','I71','I731','I738','I739','I771','I790','I792','K551','K558','K559','Z958','Z959',
                                                            '0930','4373','440','441','4431','4432','4433','4434','4435','4436','4437','4438','4439',
                                                            '4471','5571','5579','V434'],
                                                           2],
                        'hypertension uncomplicated' : [['I10','401']
                                                        ,0],
                        'hypertension complicated' : [['I11','I12','I13','I15','402','403','404','405'],
                                                      0],
                        'paralysis' : [['G041','G114','G801','G802','G81','G82','G830','G831','G832','G833','G834','G839',
                                        '3341','342','343','3440','3441','3442','3443','3444','3445','3446','3449'],
                                       7],
                        'other neurological disorders' : [['G10','G11','G12','G13','G20','G21','G22','G254','G255','G312','G318','G319','G32','G35',
                                                           'G36','G37','G40','G41','G931','G934','R470','R56',
                                                           '3319','3320','3321','3334','3335','33392','334','335','3362','340','341','345','3481',
                                                           '3483','7803','7843'],
                                                          6],
                        'chronic pulmonary disease' : [['I278','I279','J40','J41','J42','J43','J44','J45','J46','J47','J60','J61','J62','J63','J64',
                                                        'J65','J66','J67','J684','J701','J703',
                                                        '4168','4169','490','491','492','493','494','495','496','497','498','499','500','501','502',
                                                        '503','504','505','5064','5081','5088'],
                                                       3],
                        'diabetes uncomplicated' : [['E100','E101','E109','E110','E111','E119','E120','E121','E129','E130','E131','E139',
                                                     'E140','E141','E149',
                                                     '2500','2501','2502','2503'],
                                                    0],
                        'diabetes complicated' : [['E102','E103','E104','E105','E106','E107','E108','E112','E113','E114','E115','E116',
                                                   'E117','E118','E122','E123','E124','E125','E126','E127','E128','E132','E133','E134',
                                                   'E135','E136','E137','E138','E142','E143','E144','E145','E146','E147','E148',
                                                   '2504','2505','2506','2507','2508','2509'],
                                                  0],
                        'hypothyroidism' : [['E00','E01','E02','E03','E890',
                                             '2409','243','244','2461','2468'],
                                            0],
                        'renal failure' : [['I120','I131','N18','N19','N250','Z490','Z491','Z492','Z940','Z992',
                                            '40301','40311','40391','40402','40403','40412','40413','40492','40493','585','586','5880','V420','V451','V56'],
                                           5],
                        'liver disease' : [['B18','I85','I864','I982','K70','K711','K713','K714','K715','K717','K72','K73','K74','K760','K762','K763',
                                            'K764','K765','K766','K767','K768','K769','Z944',
                                            '07022','07023','07032','07033','07044','07054','0706','0709','4560','4561','4562',
                                            '570','571','5722','5723','5724','5725',
                                            '5726','5727','5728','5733','5734','5738','5739','V427'],
                                           11],
                        'peptic ulcer disease' : [['K257','K259','K267','K269','K277','K279','K287','K289',
                                                   '5317','5319','5327','5329','5337','5339','5347','5349'],
                                                  0],
                        'aids' : [['B20','B21','B22','B24',
                                   '042','043','044'],
                                  0],
                        'lymphoma' : [['C81','C82','C83','C84','C85','C88','C96','C900','C902',
                                       '200','201','202','2030','2386'],
                                      9],
                        'metastatic cancer' : [['C77','C78','C79','C80',
                                                '196','197','198','199'],
                                               12],
                        'solid tumor without metastasis' : [['C00','C01','C02','C03','C04','C05','C06','C07','C08','C09',
                                                             'C10','C11','C12','C13','C14','C15','C16','C17','C18','C19',
                                                             'C20','C21','C22','C23','C24','C25','C26','C30','C31','C32',
                                                             'C33','C34','C37','C38','C39','C40','C41','C43','C45','C46',
                                                             'C47','C48','C49','C50','C51','C52','C53','C54','C55','C56',
                                                             'C57','C58','C60','C61','C62','C63','C64','C65','C66','C67',
                                                             'C68','C69','C70','C71','C72','C73','C74','C75','C76','C97',
                                                             '140','141','142','143','144','145','146','147','148','149',
                                                             '150','151','152','153','154','155','156','157','158','159',
                                                             '160','161','162','163','164','165','166','167','168','169',
                                                             '170','171','172','174','175','176','177','178','179','180',
                                                             '181','182','183','184','185','186','187','188','189','190',
                                                             '191','192','193','194','195'],
                                                            4],
                        'rheumatoid arthritis or collagen vascular diseases' : [['L940','L941','L943','M05','M06','M08','M120','M123','M30','M310',
                                                                                 'M311','M312','M313','M32','M33',
                                                                                 'M34','M35','M45','M461','M468','M469',
                                                                                 '446','7010','7100','7101','7102','7103','7104','7108','7109','7112',
                                                                                 '714','7193','720','725','7285','72889','72930'],
                                                                                0],
                        'coagulopathy' : [['D65','D66','D67','D68','D691','D693','D694','D695','D696',
                                           '286','2871','2873','2874','2875'],
                                          3],
                        'obesity' : [['E66',
                                      '2780'],
                                     -4],
                        'weight loss' : [['E40','E41','E42','E43','E44','E45','E46','R634','R64',
                                          '260','261','262','263','7832','7994'],
                                         6],
                        'fluid and electrolyte disorders' : [['E222','E86','E87',
                                                              '2536','276'],
                                                             5],
                        'blood loss anemia' : [['D500',
                                               '2800'],
                                               -2],
                        'deficiency anemia' : [['D508','D509','D51','D52','D53',
                                               '2801','2802','2803','2804','2805','2806','2807','2808','2809','281'],
                                               -2],
                        'alcohol abuse' : [['F10','E52','G621','I426','K292','K700','K703','K709','T51','Z502','Z714','Z721',
                                            '2652','2911','2912','2913','2915','2916','2917','2918','2919','3030','3039','3050',
                                            '3575','4255','5353','5710','5711','5712','5713','980','V113'],
                                           0],
                        'drug abuse' : [['F11','F12','F13','F14','F15','F16','F18','F19','Z715','Z722',
                                         '292','304','3052','3053','3054','3055','3056','3057','3058','3059','V6542'],
                                        -7],
                        'psychoses' : [['F20','F22','F23','F24','F25','F28','F29','F302','F312','F315',
                                        '2938','295','29604','29614','29644','29654','297','298'],
                                       0],
                        'depression' : [['F204','F313','F314','F315','F32','F33','F341','F412','F432',
                                         '2962','2963','2965','3004','309','311'],
                                        -3]
}


In [31]:
condition_df = pd.DataFrame.from_dict(condition_dictionary,orient='index',columns=columns)
condition_df

,icd,weight
congestive heart failure,"[39891, 40201, 40211, 40291, 40401, 40403, 404...",7
cardiac arrhythmias,"[I441, I442, I443, I456, I459, I47, I48, I49, ...",5
valvular disease,"[A520, I05, I06, I07, I08, I091, I098, I34, I3...",-1
pulmonary circulation disorders,"[I26, I27, I280, I288, I289, 4150, 4151, 416, ...",4
peripheral vascular disorders,"[I70, I71, I731, I738, I739, I771, I790, I792,...",2
hypertension uncomplicated,"[I10, 401]",0
hypertension complicated,"[I11, I12, I13, I15, 402, 403, 404, 405]",0
paralysis,"[G041, G114, G801, G802, G81, G82, G830, G831,...",7
other neurological disorders,"[G10, G11, G12, G13, G20, G21, G22, G254, G255...",6
chronic pulmonary disease,"[I278, I279, J40, J41, J42, J43, J44, J45, J46...",3


# Extract the ICD codes and reformat for querying

Pull out the ICD codes from the DataFrame, combine into a single list, then split into a dictionary by length of code. 


In [32]:
# Extract ICD codes
condition_list_icd = condition_df['icd'].to_list()

# Flatten list
condition_list_icd = [item for sublist in condition_list_icd for item in sublist]

# Split into dictionary based on length of ICD code
condition_dictionary_icd = split_list_to_dict(condition_list_icd)


## Query Database 

Next, we'll query the database using the ICD codes, returninng a DataFrame for analysis

1) First, join bith dates for each person to condition_occurrence table, then only keep conditions that are (1) from the problem list and (2) when condition start date is on or after the birth date

2) from this joined table, select rows where the condition_source_value matches any of the condition ICD codes and return those codes for downstream analysis

In [33]:
condition_query = ("""\
                    WITH condition_start_filter AS ( 
                        SELECT  vco.* 
                        FROM omop.v_condition_occurrence AS vco 
                        LEFT JOIN omop.person AS p 
                        ON vco.person_id = p.person_id 
                        WHERE condition_type_concept_id = 32840 
                            AND (vco.condition_start_date - p.birth_datetime::date) >= 0 
                    ) 
                    SELECT DISTINCT person_id,  
                    CASE WHEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,5) IN {2} THEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,5) 
                        WHEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,4) IN {1} THEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,4) 
                        WHEN SUBSTRING(condition_source_value,1,3) IN {0} THEN SUBSTRING(condition_source_value,1,3) 
                        ELSE NULL END AS condition_source_value, 
                    condition_source_concept_vocabulary_id 
                    FROM condition_start_filter 
                    WHERE CASE 
                        WHEN SUBSTRING(condition_source_value,1,3) IN {0} THEN 1 
                        WHEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,4) IN {1} THEN 1 
                        WHEN SUBSTRING(TRANSLATE(condition_source_value,'.',''),1,5) IN {2} THEN 1 
                        ELSE 0 
                        END = 1; 
                    """).format(tuple(condition_dictionary_icd[3]),tuple(condition_dictionary_icd[4]),tuple(condition_dictionary_icd[5]))


In [34]:
column_names = ['person_id','condition_source_value','condition_source_concept_vocabulary_id']

person_condition_df = query_to_dataframe(condition_query,column_names=column_names)

### Query With Date

This query operates identically to the one above, but is restricted to a date range provides in the query. The dates are specified in lines 9 and 10 of the query:

```
and vco.condition_start_date >= '2015-01-03'  
and vco.condition_start_date < '2016-01-03' 
```

You can change the dates at the end of both of lines to set the range that you want to filter on. Dates must be formatted as YYYY-MM-DD. This filters the entire database. You should run this query instead of the one below, then continue with the notebook. 

In [31]:
condition_query_with_daterange = ("""\
                    WITH condition_start_filter AS ( 
                        SELECT vco.* 
                        FROM omop.v_condition_occurrence AS vco 
                        LEFT JOIN omop.person AS p 
                        ON vco.person_id = p.person_id 
                        WHERE condition_type_concept_id = 32840 
                            AND (vco.condition_start_date - p.birth_datetime::date) >= 0 
                            AND vco.condition_start_date >= '2015-01-03'  
                            AND vco.condition_start_date < '2016-01-03' 
                    ) 
                    SELECT DISTINCT person_id,  
                    CASE WHEN substring(translate(condition_source_value,'.',''),1,5) IN {2} THEN substring(translate(condition_source_value,'.',''),1,5) 
                        WHEN substring(translate(condition_source_value,'.',''),1,4) IN {1} THEN substring(translate(condition_source_value,'.',''),1,4) 
                        WHEN substring(condition_source_value,1,3) IN {0} THEN substring(condition_source_value,1,3) 
                        ELSE NULL END AS condition_source_value, 
                    condition_source_concept_vocabulary_id 
                    FROM condition_start_filter 
                    WHERE CASE 
                        WHEN substring(condition_source_value,1,3) IN {0} THEN 1 
                        WHEN substring(translate(condition_source_value,'.',''),1,4) IN {1} THEN 1 
                        WHEN substring(translate(condition_source_value,'.',''),1,5) IN {2} THEN 1 
                        ELSE 0 
                        END = 1; 
                    """).format(tuple(condition_dictionary_icd[3]),tuple(condition_dictionary_icd[4]),tuple(condition_dictionary_icd[5]))


In [32]:
column_names_with_daterange = ['person_id','condition_source_value','condition_source_concept_vocabulary_id']
person_condition_df = query_to_dataframe(condition_query_with_daterange,column_names_with_daterange)


In [33]:
person_condition_df.head()

,person_id,condition_source_value,condition_source_concept_vocabulary_id
0,000020DCF6C9D57FFF62D105181FDFB8,2500,ICD9CM
1,000095C6F6910B8657FAB9ABD2E8B8F6,401,ICD9CM
2,0000E9391C0DE0AAD9DC97BBFE7B2B78,2500,ICD9CM
3,0000E9391C0DE0AAD9DC97BBFE7B2B78,3483,ICD9CM
4,0000E9391C0DE0AAD9DC97BBFE7B2B78,585,ICD9CM


In [34]:
person_condition_df.shape

(355219, 3)

# Update person_condition_df for conditions listed multiple ways

There are multiple codes listed to different levels of specificity. We need to add in rows so we have records of both codes in the dataframe. Our original query returns the longer form of these codes, so we will look specifically for those, then add in the abbreviated code. 

In [35]:
# Build a dictionary that has the condtions listed

duplicate_length_codes = {'I11' : ['I110'],
                          'I13' : ['I130','I132','I131'],
                          'I12' : ['I120'],
                          'I27' : ['I278','I279'],
                          'K70' : ['K700','K703','K709'],
                          '2965' : ['29654'],
                          'G11' : ['G114'],
                          '334' : ['3341'],
                          '404' : ['40401','40402','40403','40411','40412','40413','40491','40492','40493'],
                          '403' : ['40301','40311','40391'],
                          '402' : ['40201','40211','40291'],
                         '416' : ['4168','4169'],
                         '571' : ['5710','5711','5712','5713']}

In [36]:
person_condition_df = add_new_row(person_condition_df,duplicate_length_codes)

## Check that subset on conditions correspond to ICD9, not ICD10

Check for the following conditions: V45.0, V53.3, V42.2, V43.3, V43.4, V42.0, V45.1, V56, V42.7, V11.3, V65.42

These can correspond to codes in both ICD9 and ICD10, but for Elixhauser should be ICD9 codes (per Quan 2005). Therefore, remove rows where the condition is one of those and the source vocabulary is ICD10CM. 

In [37]:
# Filter out rows where condition_source matches list and vocabulary_id is ICD10CM
person_condition_df = person_condition_df[~((person_condition_df['condition_source_value'].isin(['V450','V533','V422','V433','V434','V420','V451',
                                                                                                 'V56','V427','V113','V6542'])) & 
                   (person_condition_df['condition_source_concept_vocabulary_id'] == 'ICD10CM'))]


# Group conditions by person_id

In [38]:
# Use groupby to get a list of all conditions for each person
person_condition_grouped_df = person_condition_df.groupby('person_id')['condition_source_value'].apply(list).reset_index()


## Check for related conditions

We don't double count closely related diseases. Table 1 Footnote B of Elixhauser et al. (1998) gives the algorithm: A hierarchy was established between the following pairs of comorbidities: If both uncomplicated complicated diabetes are present, count only complicated diabetes. If both solid tumor without metastatis and metastatic cancer are present, count only metastatic cancer


In [39]:
diabetes_uncomplicated = condition_dictionary['diabetes uncomplicated'][0]
diabetes_complicated = condition_dictionary['diabetes complicated'][0]
person_condition_grouped_filtered_df = filter_related_conditions(person_condition_grouped_df, diabetes_complicated, diabetes_uncomplicated)

tumor_without_metastatis = condition_dictionary['solid tumor without metastasis'][0]
metastatic_cancer = condition_dictionary['metastatic cancer'][0]
person_condition_grouped_filtered_df = filter_related_conditions(person_condition_grouped_filtered_df, metastatic_cancer, tumor_without_metastatis)



## Calculate Elixhauser Index

In [40]:
# Add in a column for the score for each person

person_condition_grouped_filtered_df['comorbidity_score'] = person_condition_grouped_filtered_df['condition_source_value'].apply(lambda x: calculate_score(x, condition_df))